In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time

import datetime
import pytz
import calendar
import itertools
import pandas as pd

import datetime
import sys

import numpy as np
import os
import pandas as pd
import pytz

sys.path.insert(0, '../')
sys.path.insert(0, '../Optimizers')

# import utils
# from utils import plotly_figure

import itertools
import xbos_services_getter as xsg

from Optimizers.MPC.MPC import MPC
from Optimizers.MPC.MPC import Node
from DataManager.DataManager import DataManager


# from Optimizers.MPC.MPC import MPC
# from Optimizers.MPC.MPC import Node
# from DataManager.DataManager import DataManager
from Thermostat import Tstat

In [3]:
from Simulation.Simulation import SimulationMPC
from datetime import *
import xbos_services_getter as xsg

In [4]:
TZ=pytz.timezone('US/Pacific')
#start = datetime(year=2016,day=1,month=1,hour=0,tzinfo=TZ)
start = datetime.now(TZ)-timedelta(days=2)
end = datetime.now(TZ)-timedelta(days=1)
# 1 zone at a time
# (start, end): beginning and end of the simulated interval
# horizon 4h is good
building = 'avenal-movie-theatre'
horizon = '4h'
window = "15m"
lambda_val = 0.9
zone = 'hvac_zone_lobby'
building_zone_names_stub = xsg.get_building_zone_names_stub()
all_zones = xsg.get_zones(building_zone_names_stub, building)
starting_temperatures = {zone: 66. for zone in all_zones}


In [ ]:

def simulation(building, start, end, horizon, lambda_val, zone=None):
    all_zones = get_zones(building)
    use_zones = [zone] if zone is not None else all_zones
    #zonename = all_zones[0]
    print(building, start, end, horizon, lambda_val)
    results = {}
    for zonename in use_zones:
        starting_temperatures = {zone: 66. for zone in all_zones}
        # 15 minutes here is the simulated step
        res = xsg.get_mpc_simulation(optimizer_stub, building, [zonename], start, end, '15m', horizon, lambda_val, starting_temperatures)
        dates = pd.date_range(start, end, freq='15T')
        actions, temperatures = res
        # we pad the actions with an 'off'
        actions = list(actions[0][zonename].actions) + [0]
        d = {
                'time': dates,
                'actions': actions,
                'temperatures': temperatures[0][zonename].temperatures,
            }
        print({k: len(v) for k,v in d.items()})
        df = pd.DataFrame.from_dict(d)
        df = df.set_index(pd.to_datetime(df.pop('time')))
        df = df.resample('15T').max()
        results[zonename] = df
    return results

In [5]:
tstats = {}
for iter_zone in all_zones:
    print(iter_zone)
    tstats[iter_zone] = Tstat(building, iter_zone, starting_temperatures[iter_zone],  suppress_not_enough_data_error=True)



hvac_zone_lobby
hvac_zone_main_hallway
hvac_zone_room_a
hvac_zone_theater_2
hvac_zone_back_hallway
hvac_zone_room_d
hvac_zone_pegasus_hall
hvac_zone_theater_1


In [12]:

Simulation_instance = SimulationMPC(building, [zone], lambda_val,
                                        start, end, horizon, window, tstats)

{'hvac_zone_lobby':                            t_high  t_low unit
datetime                                     
2019-06-05 14:09:45-07:00    76.0   68.0    F
2019-06-05 14:24:45-07:00    76.0   68.0    F
2019-06-05 14:39:45-07:00    76.0   68.0    F
2019-06-05 14:54:45-07:00    76.0   68.0    F
2019-06-05 15:09:45-07:00    76.0   68.0    F
2019-06-05 15:24:45-07:00    76.0   68.0    F
2019-06-05 15:39:45-07:00    76.0   68.0    F
2019-06-05 15:54:45-07:00    76.0   68.0    F
2019-06-05 16:09:45-07:00    76.0   68.0    F
2019-06-05 16:24:45-07:00    76.0   68.0    F
2019-06-05 16:39:45-07:00    76.0   68.0    F
2019-06-05 16:54:45-07:00    77.3   68.0    F
2019-06-05 17:09:45-07:00    78.0   68.0    F
2019-06-05 17:24:45-07:00    78.0   68.0    F
2019-06-05 17:39:45-07:00    78.0   68.0    F
2019-06-05 17:54:45-07:00    78.0   68.0    F
2019-06-05 18:09:45-07:00    78.0   68.0    F
2019-06-05 18:24:45-07:00    78.0   68.0    F
2019-06-05 18:39:45-07:00    78.0   68.0    F
2019-06-05 18:

Exception: Bad comfortband given. Does not have valid start or/and end time.

In [10]:
print(start.replace(microsecond=0))

2019-06-05 14:09:45-07:00


In [11]:
print(end.replace(microsecond=0))

2019-06-06 14:09:45-07:00
